# Ulanowicz et al. (2009) - Network Analysis Validation

## Paper: "Quantifying sustainability: Resilience, efficiency and the return of information theory"

This notebook demonstrates the Cone Spring ecosystem networks and metrics from the paper.

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Load the Cone Spring Ecosystem Data

In [ ]:
# Load both network configurations
data_path = Path('../data/ecosystem_samples')

with open(data_path / 'cone_spring_original.json', 'r') as f:
    original_data = json.load(f)

with open(data_path / 'cone_spring_eutrophicated.json', 'r') as f:
    eutrophicated_data = json.load(f)

# Extract flow matrices
original_flows = np.array(original_data['flows'])
eutrophicated_flows = np.array(eutrophicated_data['flows'])
node_names = original_data['nodes']

print("Node names:", node_names)
print("\nOriginal network shape:", original_flows.shape)
print("Eutrophicated network shape:", eutrophicated_flows.shape)

## 2. Display Flow Matrices

Flow convention: `flow[i,j]` = flow from node i to node j (kcal m⁻² y⁻¹)

In [ ]:
# Create DataFrames for better display
df_original = pd.DataFrame(original_flows, 
                           index=node_names, 
                           columns=node_names)

df_eutrophicated = pd.DataFrame(eutrophicated_flows,
                                index=node_names,
                                columns=node_names)

# Display original matrix
print("ORIGINAL CONE SPRING ECOSYSTEM")
print("=" * 40)
print("\nFlow Matrix (kcal m⁻² y⁻¹):")
display(df_original.style.format("{:.0f}").background_gradient(cmap='YlOrRd'))

print("\nTotal Internal Flows:", original_flows.sum())
print("Row sums (outputs):", original_flows.sum(axis=1))
print("Column sums (inputs):", original_flows.sum(axis=0))

In [ ]:
# Display eutrophicated matrix
print("EUTROPHICATED CONE SPRING ECOSYSTEM")
print("=" * 40)
print("\nFlow Matrix (kcal m⁻² y⁻¹):")
display(df_eutrophicated.style.format("{:.0f}").background_gradient(cmap='YlOrRd'))

print("\nTotal Internal Flows:", eutrophicated_flows.sum())
print("Row sums (outputs):", eutrophicated_flows.sum(axis=1))
print("Column sums (inputs):", eutrophicated_flows.sum(axis=0))

In [ ]:
# Show the difference (eutrophication effect)
df_difference = df_eutrophicated - df_original

print("EUTROPHICATION EFFECT (Difference)")
print("=" * 40)
print("\nAdded 8000 kcal m⁻² y⁻¹ to pathway Plants→Detritus→Bacteria")
print("\nFlow Changes:")
display(df_difference.style.format("{:.0f}").background_gradient(cmap='coolwarm'))

print("\nTotal flow increase:", df_difference.values.sum(), "kcal m⁻² y⁻¹")

## 3. Published Metrics from Ulanowicz et al. (2009)

These are the metrics as reported in the paper (Figure 5 and Figure 6):

In [ ]:
# Create comparison table of published metrics
published_metrics = pd.DataFrame({
    'Metric': [
        'Relative Ascendency (α)',
        'Optimal α',
        'Distance from Optimal',
        'System Status',
        'Marginal Contribution Pattern',
        'Sustainability Assessment'
    ],
    'Original Network': [
        '0.418',
        '0.460',
        '0.042 (below)',
        'Below optimal',
        'Main pathway > 1, Parallel < 1',
        'Can still grow and develop'
    ],
    'Eutrophicated Network': [
        '0.529',
        '0.460',
        '0.069 (above)',
        'Above optimal',
        'Main pathway < 1, Parallel > 1',
        'Excess ascendency, reduced reserve'
    ]
})

print("PUBLISHED METRICS FROM PAPER")
print("=" * 60)
display(published_metrics.style.set_properties(**{'text-align': 'left'}))

## 4. Network Visualization

In [ ]:
def create_network_graph(flow_matrix, node_names, title, ax):
    """Create a network visualization from flow matrix."""
    G = nx.DiGraph()
    
    # Add nodes
    for i, name in enumerate(node_names):
        G.add_node(name)
    
    # Add edges with weights
    for i in range(len(flow_matrix)):
        for j in range(len(flow_matrix)):
            if flow_matrix[i, j] > 0:
                G.add_edge(node_names[i], node_names[j], 
                          weight=flow_matrix[i, j])
    
    # Position nodes in a circle
    pos = nx.circular_layout(G)
    
    # Draw network
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', 
                          node_size=2000, ax=ax)
    
    # Draw edges with varying thickness based on flow
    edges = G.edges()
    weights = [G[u][v]['weight'] for u, v in edges]
    max_weight = max(weights) if weights else 1
    
    # Normalize edge widths
    edge_widths = [3 * w / max_weight for w in weights]
    
    nx.draw_networkx_edges(G, pos, width=edge_widths, 
                          edge_color='gray', arrows=True,
                          arrowsize=20, arrowstyle='->', 
                          connectionstyle='arc3,rad=0.1', ax=ax)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=10, 
                           font_weight='bold', ax=ax)
    
    # Add edge labels with flow values
    edge_labels = {(u, v): f"{G[u][v]['weight']:.0f}" 
                  for u, v in edges}
    nx.draw_networkx_edge_labels(G, pos, edge_labels, 
                                 font_size=8, ax=ax)
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.axis('off')

# Create side-by-side network visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

create_network_graph(original_flows, node_names, 
                    "Original Cone Spring Network\n(α = 0.418)", ax1)
create_network_graph(eutrophicated_flows, node_names, 
                    "Eutrophicated Cone Spring Network\n(α = 0.529)", ax2)

plt.tight_layout()
plt.show()

## 5. Key Information Theory Formulas

The core formulas from Ulanowicz et al. (2009) using **natural logarithms**:

In [ ]:
# Display the key formulas
formulas = pd.DataFrame({
    'Metric': [
        'Development Capacity (C)',
        'Ascendency (A)',
        'Reserve/Overhead (Φ)',
        'Relative Ascendency (α)',
        'Robustness (R)',
        'Fundamental Relationship'
    ],
    'Formula': [
        'C = -Σ(Tᵢⱼ × ln(Tᵢⱼ/T..))',
        'A = Σ(Tᵢⱼ × ln(Tᵢⱼ×T.. / (Tᵢ.×T.ⱼ)))',
        'Φ = C - A',
        'α = A/C',
        'R = T.. × F where F = -(e/ln(e)) × αᵇ × ln(αᵇ)',
        'C = A + Φ (must always hold)'
    ],
    'Equation #': [
        'Eq. 11',
        'Eq. 12',
        'Eq. 13',
        'Derived',
        'Eq. 17',
        'Eq. 14'
    ],
    'Units': [
        'flow-nats',
        'flow-nats',
        'flow-nats',
        'dimensionless',
        'flow units',
        '-'
    ]
})

print("KEY INFORMATION THEORY FORMULAS")
print("=" * 60)
display(formulas.style.set_properties(**{'text-align': 'left'}))

## 6. Window of Viability Analysis

In [ ]:
# Create Window of Viability visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Define the window boundaries
alpha_values = np.linspace(0, 1, 100)
optimal_alpha = 0.460
lower_bound = 0.20
upper_bound = 0.60

# Robustness function (simplified)
b = 1.288
robustness = lambda a: -(np.e/np.log(np.e)) * a**b * np.log(a**b) if a > 0 else 0
robustness_values = [robustness(a) if a > 0.001 else 0 for a in alpha_values]

# Plot robustness curve
ax.plot(alpha_values, robustness_values, 'b-', linewidth=2, label='Robustness')

# Mark the window of viability
ax.axvspan(lower_bound, upper_bound, alpha=0.2, color='green', label='Window of Viability')
ax.axvline(optimal_alpha, color='red', linestyle='--', label=f'Optimal α = {optimal_alpha}')

# Mark the two networks
ax.axvline(0.418, color='blue', linestyle=':', label='Original (α = 0.418)')
ax.axvline(0.529, color='orange', linestyle=':', label='Eutrophicated (α = 0.529)')

# Labels and formatting
ax.set_xlabel('Relative Ascendency (α)', fontsize=12)
ax.set_ylabel('Robustness', fontsize=12)
ax.set_title('Window of Viability and Network Positions', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='best')

# Add text annotations
ax.text(0.1, 0.5, 'Too\nChaotic', ha='center', fontsize=10, color='red')
ax.text(0.4, 0.9, 'Viable\nSystems', ha='center', fontsize=10, color='green')
ax.text(0.8, 0.5, 'Too\nRigid', ha='center', fontsize=10, color='red')

plt.tight_layout()
plt.show()

## 7. Exogenous Flows and Complete System

In [ ]:
# Display exogenous flows
def display_exogenous_flows(data, title):
    metadata = data['metadata']
    
    # Create DataFrame for exogenous inputs
    inputs_df = pd.DataFrame([
        metadata.get('exogenous_inputs', {})
    ]).T
    inputs_df.columns = ['Input Flow']
    inputs_df.index = [idx.replace('to_', '').title() for idx in inputs_df.index]
    
    # Create DataFrame for exogenous outputs
    outputs_df = pd.DataFrame([
        metadata.get('exogenous_outputs', {})
    ]).T
    outputs_df.columns = ['Export']
    outputs_df.index = [idx.replace('from_', '').title() for idx in outputs_df.index]
    
    # Create DataFrame for dissipations
    dissipations_df = pd.DataFrame([
        metadata.get('dissipations', {})
    ]).T
    dissipations_df.columns = ['Respiration']
    dissipations_df.index = [idx.title() for idx in dissipations_df.index]
    
    # Combine all
    complete_df = pd.concat([inputs_df, outputs_df, dissipations_df], axis=1)
    complete_df = complete_df.fillna(0)
    
    print(f"\n{title}")
    print("=" * 40)
    display(complete_df.style.format("{:.0f}").background_gradient(cmap='YlOrRd'))
    
    print(f"\nTotal Inputs: {inputs_df.sum().values[0]:.0f} kcal m⁻² y⁻¹")
    print(f"Total Exports: {outputs_df.sum().values[0]:.0f} kcal m⁻² y⁻¹")
    print(f"Total Respiration: {dissipations_df.sum().values[0]:.0f} kcal m⁻² y⁻¹")
    
    return complete_df

# Display for both networks
orig_exo = display_exogenous_flows(original_data, "ORIGINAL NETWORK - EXOGENOUS FLOWS")
eutr_exo = display_exogenous_flows(eutrophicated_data, "EUTROPHICATED NETWORK - EXOGENOUS FLOWS")

## 8. Summary of Key Findings

### From the Paper (Ulanowicz et al. 2009):

1. **Optimal Relative Ascendency**: α_opt = 0.4596 (derived from Window of Vitality)

2. **Window of Viability**: 0.20 < α < 0.60
   - Below 0.20: System too chaotic (insufficient organization)
   - Above 0.60: System too rigid (over-organized)

3. **Cone Spring Original** (α = 0.418):
   - Below optimal → System can still grow and develop
   - Marginal contributions: Main pathway flows > 1, parallel flows < 1
   - System would benefit from strengthening the main pathway

4. **Cone Spring Eutrophicated** (α = 0.529):
   - Above optimal → System has excess ascendency over reserve
   - Marginal contributions inverted: Main pathway < 1, parallel flows > 1
   - System needs more diversity and redundancy

5. **Key Insight**: Evolution favors moderation - systems need balance between efficiency (ascendency) and resilience (reserve) for long-term sustainability.

In [ ]:
# Create final comparison summary
print("ECOSYSTEM TRANSFORMATION SUMMARY")
print("=" * 60)
print(f"\nEutrophication Effect:")
print(f"  • Added 8000 kcal m⁻² y⁻¹ to Plants→Detritus→Bacteria pathway")
print(f"  • Relative Ascendency increased: 0.418 → 0.529 (+26.6%)")
print(f"  • System moved from below optimal to above optimal")
print(f"  • Changed from growth-favorable to diversity-needed state")
print(f"\nSustainability Implications:")
print(f"  • Original: Healthy system with room for development")
print(f"  • Eutrophicated: Over-constrained system, vulnerable to shocks")
print(f"  • Recommendation: Increase pathway diversity to restore balance")